### 构建一个模块的层级包

文件__init__.py的目的是要包含不同运行级别的包的可选的初始化代码

举个例子，__init__.py能够用来自动加载子模块:

In [ ]:
# graphics/formats/__init__.py
from . import jpg
from . import png

### 控制模块被全部导入的内容

当使用 from module import * 时，明确列出导出内容

In [ ]:
# somemodule.py
def spam():
    pass

def grok():
    pass

blah = 42
# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']

### 相对路径名导入

In [ ]:
# mypackage/A/spam.py
from . import grok

# mypackage/A/spam.py
from ..B import bar

from . import grok # OK
import .grok # ERROR

% python3 mypackage/A/spam.py # Relative imports fail

% python3 -m mypackage.A.spam # Relative imports work

### 将模块分割成多个文件

使用 __init__.py 文件来将每部分粘合在一起

In [ ]:
# __init__.py
from .a import A
from .b import B

# 延迟导入
# 延迟加载的主要缺点是继承和类型检查可能会中断
# __init__.py
def A():
    from .a import A
    return A()

def B():
    from .b import B
    return B()

import mymodule
a = mymodule.A()
a.spam()

### 利用命名空间导入目录分散的代码
在统一不同的目录里统一相同的命名空间，但是要删去用来将组件联合起来的__init__.py文件

```
foo-package/
    spam/
        blah.py

bar-package/
    spam/
        grok.py
```
两个目录中的spam共同组成了同一个命名空间

In [ ]:
import sys

sys.path.extend(['foo-package', 'bar-package'])
import spam.blah
import spam.grok
import spam
print(spam.__path__)
# _NamespacePath(['foo-package/spam', 'bar-package/spam'])

### 重新加载模块

In [ ]:
import spam
import imp
imp.reload(spam)

# 该 load 会被重置
import spam
# 该 load 不会被重置
from spam import grok

### 运行目录
```
myapplication/
    spam.py
    bar.py
    grok.py
    __main__.py
```

In [ ]:
% python3 myapplication

### 读取位于包中的数据文件
```
mypackage/
    __init__.py
    somedata.dat
    spam.py
```

In [ ]:
# spam.py
import pkgutil
# 返回 原始内容的字节字符串
data = pkgutil.get_data(__package__, 'somedata.dat')

### 将文件夹加入到sys.path
- 使用PYTHONPATH环境变量
- 创建一个.pth文件

In [ ]:
import sys
from os.path import abspath, join, dirname
sys.path.insert(0, join(abspath(dirname(__file__)), 'src'))

### 通过字符串名导入模块

In [ ]:
import importlib
# Same as 'import math'
math = importlib.import_module('math')

# Same as 'from . import b'
b = importlib.import_module('.b', __package__)

### 通过钩子远程加载模块

### 导入模块的同时修改模块

In [ ]:
from functools import wraps
from postimport import when_imported

def logged(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Calling', func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return wrapper

# Example
@when_imported('math')
def add_logging(mod):
    mod.cos = logged(mod.cos)
    mod.sin = logged(mod.sin)